In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
import joblib
import time
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.utils.class_weight import compute_class_weight
from torch.amp import autocast, GradScaler
from torch.utils.data import TensorDataset, DataLoader

In [2]:
# Enable cuDNN benchmarking for faster training
torch.backends.cudnn.benchmark = True

In [3]:
# 1. Load dataset and remove rows with '-' in the tag column
df = pd.read_csv("figma_full_dataset.csv")
df = df[~df['tag'].str.contains(r'(-|CNX|ADDRESS|ASIDE|CANVAS|CITE|DD|DL|DT|ICON|S|SUP|VECTOR|DEL|LEGEND|BDI|LOGO|OBJECT|OPTGROUP)', regex=True)]

C:\Users\AOZ\AppData\Local\Temp\ipykernel_25352\3865457134.py:3: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df = df[~df['tag'].str.contains(r'(-|CNX|ADDRESS|ASIDE|CANVAS|CITE|DD|DL|DT|ICON|S|SUP|VECTOR|DEL|LEGEND|BDI|LOGO|OBJECT|OPTGROUP)', regex=True)]


In [4]:
# 2. Separate features and target
y = df["tag"]
X = df.drop(columns=["tag"])

In [5]:
# 3. Identify categorical and continuous columns
categorical_cols = ['type','parent_tag','parent_tag_html']
continuous_cols = [col for col in X.columns if col not in categorical_cols]

In [6]:
# Process categorical features with OneHotEncoder instead of LabelEncoder
X[categorical_cols] = X[categorical_cols].astype(str).fillna('unknown')
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
X_cat_encoded = ohe.fit_transform(X[categorical_cols])
joblib.dump(ohe, "ohe_encoder.pkl")

['ohe_encoder.pkl']

In [7]:
# Better missing value handling with imputer
imputer = SimpleImputer(strategy='median')
X_continuous_imputed = imputer.fit_transform(X[continuous_cols])
joblib.dump(imputer, "imputer.pkl")

['imputer.pkl']

In [8]:
# Scale continuous features
scaler = StandardScaler()
X_continuous_scaled = scaler.fit_transform(X_continuous_imputed)
joblib.dump(scaler, "scaler.pkl")

['scaler.pkl']

In [9]:
# Combine one-hot encoded categorical features with scaled continuous features
X_processed = np.concatenate([X_cat_encoded, X_continuous_scaled], axis=1)

In [10]:
# 4. Encode target labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
joblib.dump(label_encoder, "label_encoder.pkl")

['label_encoder.pkl']

In [11]:
from collections import Counter

# Count occurrences of each class
class_counts = Counter(y_encoded)

# Find classes with only 1 sample
rare_classes = [cls for cls, count in class_counts.items() if count < 2]

# Duplicate rare class samples
for cls in rare_classes:
    idx = np.where(y_encoded == cls)[0][0]  # Get the index of the rare sample
    original_class_name = label_encoder.inverse_transform([cls])[0]  # Convert back to original label
    print(f"Duplicating class '{original_class_name}' (only 1 sample present).")

    X_processed = np.vstack([X_processed, X_processed[idx]])  # Duplicate features
    y_encoded = np.append(y_encoded, y_encoded[idx])  # Duplicate label

Duplicating class 'TEXTAREA' (only 1 sample present).


In [12]:
# 5. Train/test split - remove stratification if there are classes with too few samples
unique_counts = np.unique(y_encoded, return_counts=True)
min_samples = min(unique_counts[1])

if min_samples < 2:
    print(f"Warning: The least populated class has only {min_samples} sample(s). Removing stratification.")
    X_train, X_test, y_train, y_test = train_test_split(
        X_processed, y_encoded, test_size=0.2, random_state=42
    )
else:
    X_train, X_test, y_train, y_test = train_test_split(
        X_processed, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
    )

In [13]:
# Move GPU setup earlier
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [14]:
# Convert data to tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

In [15]:
# Create dataset and dataloaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(
    train_dataset, 
    batch_size=256,  # Larger batch size
    shuffle=True, 
    num_workers=4,   # Parallel loading
    pin_memory=True  # Faster data transfer to GPU
)

In [16]:
# Compute class weights
print("Computing class weights...")
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights = torch.tensor(class_weights, dtype=torch.float32).to(device)

Computing class weights...


In [17]:
# 6. Define improved model architecture with proper input size
class ImprovedTagClassifier(nn.Module):
    def __init__(self, input_size, output_size, dropout_rate=0.3):
        super(ImprovedTagClassifier, self).__init__()
        self.fc1 = nn.Linear(input_size, 256)
        self.bn1 = nn.BatchNorm1d(256)
        self.fc2 = nn.Linear(256, 256)
        self.bn2 = nn.BatchNorm1d(256)
        self.fc3 = nn.Linear(256, 128)
        self.bn3 = nn.BatchNorm1d(128)
        self.fc4 = nn.Linear(128, output_size)
        self.dropout = nn.Dropout(dropout_rate)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.dropout(self.relu(self.bn1(self.fc1(x))))
        x = self.dropout(self.relu(self.bn2(self.fc2(x))))
        x = self.dropout(self.relu(self.bn3(self.fc3(x))))
        logits = self.fc4(x)
        return logits

In [18]:
# Initialize model
print("Initializing model...")
input_size = X_train.shape[1]
output_size = len(label_encoder.classes_)
model = ImprovedTagClassifier(input_size, output_size).to(device)

Initializing model...


In [19]:
# 7. Define loss function and optimizer
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5, factor=0.5)

In [20]:
# 8. Setup mixed precision training
scaler = GradScaler(device='cuda' if torch.cuda.is_available() else 'cpu')

In [21]:
# 9. Training loop with timing and early stopping
print("Starting training...")
best_loss = float('inf')
patience = 10
counter = 0
early_stop = False
start_time = time.time()

num_epochs = 200
for epoch in range(num_epochs):
    epoch_start = time.time()
    model.train()
    epoch_loss = 0
    
    for batch_X, batch_y in train_loader:
        # Move batch to device
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        
        optimizer.zero_grad()
        
        # Use mixed precision for faster training
        with torch.amp.autocast('cuda', enabled=device.type=='cuda'):
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
        
        # Scale gradients and optimize
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        
        epoch_loss += loss.item()
    
    avg_loss = epoch_loss / len(train_loader)
    scheduler.step(avg_loss)
    
    epoch_time = time.time() - epoch_start
    if (epoch + 1) % 5 == 0:
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}, Time: {epoch_time:.2f}s")
    
    # Early stopping
    if avg_loss < best_loss:
        best_loss = avg_loss
        counter = 0
        torch.save(model.state_dict(), "best_tag_classifier.pth")
    else:
        counter += 1
        if counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            early_stop = True
    
    if early_stop:
        break

Starting training...
Epoch [5/200], Loss: 0.9440, Time: 14.05s
Epoch [10/200], Loss: 0.8067, Time: 13.82s
Epoch [15/200], Loss: 0.7448, Time: 13.95s
Epoch [20/200], Loss: 0.7072, Time: 14.16s
Epoch [25/200], Loss: 0.6880, Time: 13.44s
Epoch [30/200], Loss: 0.6668, Time: 14.04s
Epoch [35/200], Loss: 0.6469, Time: 13.47s
Epoch [40/200], Loss: 0.6406, Time: 20.73s
Epoch [45/200], Loss: 0.6344, Time: 13.59s
Epoch [50/200], Loss: 0.6319, Time: 14.62s
Epoch [55/200], Loss: 0.6350, Time: 16.24s
Epoch [60/200], Loss: 0.6192, Time: 13.28s
Epoch [65/200], Loss: 0.6102, Time: 13.21s
Epoch [70/200], Loss: 0.5782, Time: 13.23s
Epoch [75/200], Loss: 0.5692, Time: 14.06s
Epoch [80/200], Loss: 0.5674, Time: 13.22s
Epoch [85/200], Loss: 0.5497, Time: 13.23s
Epoch [90/200], Loss: 0.5441, Time: 13.06s
Epoch [95/200], Loss: 0.5466, Time: 13.48s
Epoch [100/200], Loss: 0.5342, Time: 13.13s
Epoch [105/200], Loss: 0.5478, Time: 13.07s
Epoch [110/200], Loss: 0.5257, Time: 13.20s
Epoch [115/200], Loss: 0.5332, 

In [22]:
# Save the trained model
torch.save(model.state_dict(), "tag_classifier.pth")
total_time = time.time() - start_time
print(f"Total training time: {total_time:.2f} seconds")

Total training time: 2359.49 seconds


In [23]:
# 10. Evaluation on the test set
print("Evaluating model...")
model.load_state_dict(torch.load("best_tag_classifier.pth"))  # Load best model
model.eval()

# Process test data in batches for memory efficiency
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=256)

all_predictions = []
all_labels = []

with torch.no_grad():
    for batch_X, batch_y in test_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        outputs = model(batch_X)
        _, predictions = torch.max(outputs, 1)
        all_predictions.extend(predictions.cpu().numpy())
        all_labels.extend(batch_y.cpu().numpy())

y_pred = np.array(all_predictions)
y_test_np = np.array(all_labels)

accuracy = accuracy_score(y_test_np, y_pred)
print(f"\nAccuracy: {accuracy:.4f}")

Evaluating model...


C:\Users\AOZ\AppData\Local\Temp\ipykernel_25352\762082746.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_tag_classifier.pth"))  #


Accuracy: 0.3965


In [24]:
print("\nClassification Report:")
print(classification_report(
    y_test_np, 
    y_pred,
    labels=np.unique(y_test_np),
    target_names=label_encoder.inverse_transform(np.unique(y_test_np))
))


Classification Report:
              precision    recall  f1-score   support

           A       1.00      1.00      1.00     17918
        ABBR       0.96      1.00      0.98       490
     ARTICLE       0.08      0.84      0.15       350
           B       0.10      0.84      0.18       371
  BLOCKQUOTE       0.00      0.00      0.00         1
        BODY       1.00      1.00      1.00       178
      BUTTON       0.49      0.61      0.54      4932
     CAPTION       0.48      1.00      0.65        70
      CENTER       0.00      0.00      0.00         1
        CODE       0.00      1.00      0.00         1
         DIV       0.99      0.17      0.29    104578
          EM       0.03      0.90      0.05        41
  FIGCAPTION       0.02      0.98      0.05       131
      FIGURE       0.04      0.44      0.08       195
      FOOTER       0.03      0.75      0.05       212
        FORM       0.01      0.97      0.03        79
       FRONT       1.00      1.00      1.00         2
   

In [25]:
# Print feature importances from the model
print("\nAnalyzing feature importance...")
with torch.no_grad():
    weights = model.fc1.weight.cpu().numpy()
    importance = np.abs(weights).mean(axis=0)
    
    # Get feature names (both categorical encoded and continuous)
    cat_feature_names = ohe.get_feature_names_out(categorical_cols)
    all_feature_names = np.concatenate([cat_feature_names, np.array(continuous_cols)])
    
    feature_importance = list(zip(all_feature_names, importance))
    feature_importance.sort(key=lambda x: x[1], reverse=True)
    
    print("\nTop 30 most important features:")
    for feature, imp in feature_importance[:30]:
        print(f"{feature}: {imp:.4f}")


Analyzing feature importance...

Top 30 most important features:
sibling_count: 0.6167
aspect_ratio: 0.5572
parent_tag_html_UL: 0.4015
parent_tag_html_SECTION: 0.3892
parent_tag_html_ARTICLE: 0.3870
width: 0.3669
parent_tag_html_HEADER: 0.3544
parent_tag_html_NAV: 0.3521
parent_tag_html_LI: 0.3306
parent_tag_html_TR: 0.3179
parent_tag_html_MAIN: 0.3076
parent_tag_html_SPAN: 0.3057
type_LINK_UNFURL: 0.2920
parent_tag_html_BUTTON: 0.2881
parent_tag_RECTANGLE: 0.2861
parent_tag_html_TD: 0.2818
parent_tag_html_DIV: 0.2783
parent_tag_html_P: 0.2749
type_LINE: 0.2726
parent_tag_html_TABLE: 0.2708
type_RECTANGLE: 0.2499
type_GROUP: 0.2392
parent_tag_html_FOOTER: 0.2373
parent_tag_html_FIGURE: 0.2369
parent_tag_html_TBODY: 0.2347
parent_tag_html_TH: 0.2326
parent_tag_LINK_UNFURL: 0.2100
parent_tag_GROUP: 0.2088
type_TEXT: 0.2085
parent_tag_html_FORM: 0.2080
